In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("MCK.csv")

In [ ]:
df

,Acceleration,Velocity,Position,Force
0,0.160962,-0.482996,45.158492,42.924609
1,0.155914,-0.327082,44.831410,55.068349
2,0.150866,-0.176215,44.655195,66.607411
3,0.145818,-0.030397,44.624799,77.556175
4,0.140771,0.110374,44.735173,87.929025
...,...,...,...,...
99,-0.034629,0.085888,183.456690,10.286581
100,-0.033274,0.052614,183.509304,8.326980
101,-0.031919,0.020696,183.530000,6.420293
102,-0.030563,-0.009868,183.520132,4.565424


In [ ]:
# independent values
X = df[['Acceleration','Velocity', 'Position']]

In [ ]:
# dependent values
y = df['Force']

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X,y)

LinearRegression()